<a href="https://colab.research.google.com/github/EmilSeyfullayev/Mercedes/blob/main/1_UpdatingDatabaseParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# #About first threee cells
# #In order to use colab with google drive with no authentication each time you have run this cell
# #Then you will have to run next cell to copy client_id, client secret and refresh_token
# #once you did thah you paste them in third cell and by running third cell only you dont need to authenticate each time
# # while third cell is run, adc.json file is created
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
# !cat adc.json

{
  "client_id": "32555940559.apps.googleusercontent.com",
  "client_secret": "ZmssLNjJy2998hD4CTg2ejr2",
  "refresh_token": "1//06-zowQDCYmCSCgYIARAAGAYSNwF-L9IrcEofp20bnjkSzrx00j3TuCmLeg8LB6mp-N4CSaC6I8DD0791tBo6d-ocjVrQ_7w-1OE",
  "revoke_uri": "https://accounts.google.com/o/oauth2/revoke",
  "scopes": [
    "openid",
    "https://www.googleapis.com/auth/userinfo.email",
    "https://www.googleapis.com/auth/cloud-platform",
    "https://www.googleapis.com/auth/appengine.admin",
    "https://www.googleapis.com/auth/compute",
    "https://www.googleapis.com/auth/accounts.reauth",
    "https://www.googleapis.com/auth/drive"
  ],
  "token_uri": "https://oauth2.googleapis.com/token",
  "type": "authorized_user"
}

In [ ]:
# #50 pages is around for 1 day
first_page_to_be_parsed = 0
last_page_to_be_parsed = 2

!pip install -U -q PyDrive

import httplib2
import json

from google.colab import auth
from oauth2client import GOOGLE_REVOKE_URI, GOOGLE_TOKEN_URI, client
from oauth2client.client import GoogleCredentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

auth_key = {
  "client_id": "32555940559.apps.googleusercontent.com",
  "client_secret": "ZmssLNjJy2998hD4CTg2ejr2",
  "refresh_token": "1//06-zowQDCYmCSCgYIARAAGAYSNwF-L9IrcEofp20bnjkSzrx00j3TuCmLeg8LB6mp-N4CSaC6I8DD0791tBo6d-ocjVrQ_7w-1OE"
}

credentials = client.OAuth2Credentials(
    access_token=None,
    client_id=auth_key['client_id'],
    client_secret=auth_key['client_secret'],
    refresh_token=auth_key['refresh_token'],
    token_expiry=None,
    token_uri=GOOGLE_TOKEN_URI,
    user_agent=None,
    revoke_uri=GOOGLE_REVOKE_URI)

credentials.refresh(httplib2.Http())
credentials.authorize(httplib2.Http())
cred = json.loads(credentials.to_json())
cred['type'] = 'authorized_user'

with open('adc.json', 'w') as outfile:
  json.dump(cred, outfile)

from google.colab import auth
auth.authenticate_user()

from bs4 import BeautifulSoup
import requests
import sqlite3
import pandas as pd
import numpy as np
from datetime import timezone, tzinfo
import pytz
import datetime         # datetime.datetime.now()
import time             # time.sleep(int(secs))

HOST = "https://turbo.az/"
HEADERS = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"
}


####################################################################################
list_of_links = []
dates_times = []
vip_premium = []
loans = []
barters = []

for page_number in range(first_page_to_be_parsed, last_page_to_be_parsed, 1):
  URL = "https://turbo.az/autos?page=" + str(page_number+1) + "&q%5Bmake%5D%5B%5D=" + str(4)

  requested_html = requests.get(URL, headers=HEADERS).content
  parsed_page = BeautifulSoup(requested_html, "html.parser")

  for product_section in parsed_page.find_all("div", "products"):
    # try:
    product_section.find_previous_sibling("div", {"class":"products-title"}).find_all("p", {"class":"products-title-amount"})
    # except AttributeError:
    #   pass
    # else:

      #this is for parsing links in third section
      for tag_consisiting_of_href in product_section.find_all("a", class_="products-i-link"):
        list_of_links.append(HOST+tag_consisiting_of_href.get("href"))

      #this is for parsing dates and times in third section's each of the product (24)
      # for date_and_time in product_section.findChildren("div", recursive=False):
        # try:
        #   dates_times.append(date_and_time.find_all("div", class_="products-bottom")[0].text.split(", ")[1])
        
        # except IndexError:
        #   pass
        
        # else:

        #   dates_times.append("Qeyd olunmayıb")

      #this is for vip, premium, etc
      for category in product_section.findChildren("div", recursive=False):
        vip_premium.append(category.get("class"))

      #this is for parsing loan availability information
      for loan in product_section.findChildren("div", recursive=False):
        if loan.find_all('div', class_='products-loan')==[]:
          loans.append(0)
        else:
          loans.append(1)

      #this is for parsing barter availability information
      for barter in product_section.findChildren("div", recursive=False):
        if barter.find_all('div', class_='products-barter')==[]:
          barters.append(0)
        else:
          barters.append(1)

      time.sleep(2) # in this place it will print out ok 8 times (as there are four sections)
      print("Parsed page number is "+str(page_number+1)+" out of "+str(last_page_to_be_parsed))
    


#All should be same length
print(len(list_of_links))
print(len(dates_times))
print(len(vip_premium))
print(len(loans))
print(len(barters))


##Compare number of parsed links to number of them existing in db
import gspread_dataframe as gd
import gspread as gs
gc = gs.authorize(GoogleCredentials.get_application_default())

ws = gc.open("MercedesDataBase").worksheet("sheet_1")
db_data = gd.get_as_dataframe(ws)

existed_in_db_check = []

for link in list_of_links:
  if link not in list(db_data['Link']):
    existed_in_db_check.append(0)
  elif link in list(db_data['Link']):
    existed_in_db_check.append(1)

print("Number of parsed links is: " + str(len(barters)) + ", number of them existing in db is: " + str(sum(existed_in_db_check))) 

IndentationError: ignored

In [ ]:
list_of_links

[]

In [ ]:
descriptions = []
number_of_looks = []
productS_id = []
phone_book = []
avtosalon_name = []
extras = []
product_text = []
location = []
time_parsed = []
date_parsed = []

def car_properties_parser():

  for link in list_of_links:

    requested_html = requests.get(link, headers=HEADERS).content
    parsed_page = BeautifulSoup(requested_html, "html.parser")

    try:


      #this is for 14 important descriptions of a car
      for main_description_division in parsed_page.find_all("div", class_="product-properties-value")[0:14]: #14 in order not to consider Barter and kredit
        descriptions.append(main_description_division.text)

      #this is for parsing number of looks
      looks_quantity = parsed_page.find_all("div", {"class":"product-statistics"})[0].find_all("p")[0].text.split(" ")[2]
      number_of_looks.append(looks_quantity)

      #this is for parsing products id
      product_id = parsed_page.find_all("div", {"class":"product-statistics"})[0].find_all("p")[2].text.split(" ")[2]
      productS_id.append(product_id)

      #for parsing product text
      try:
        product_text.append(parsed_page.find(class_="product-text").text)
      except AttributeError:
        product_text.append("No text for product")

      #for parsing extras
      try:
        extras.append(parsed_page.find("div", class_="product-extras").text)
      except AttributeError:
        extras.append("No extras for product")

      #this is for parsing phone numbers

      try:
        phone_book.append(parsed_page.find(class_='phone').text)

      except AttributeError:
        phones = []
        for i in parsed_page.find_all(class_='shop-contact--phones-i'):
          phones.append(i.text)
        
        phone_book.append(phones)

      
      #for avtosalon name
      
      try:
        avtosalon_name.append(parsed_page.find(class_="shop--title").text)
      except AttributeError:
        avtosalon_name.append(0)

      try:
        location.append(parsed_page.find(class_="shop-contact--location-map shop--open-location-modal").find("img").get('src').split("markers=")[1].split("&size")[0])
      except AttributeError:
        location.append(0)

      date_parsed.append(datetime.datetime.now(pytz.timezone("Asia/Baku")).strftime("%H:%M:%S"))
      time_parsed.append(datetime.datetime.now(pytz.timezone("Asia/Baku")).strftime("%Y-%m-%d"))

      print("Parsed link number is " + str(list_of_links.index(link)+1)+ " out of "+ str(len(list_of_links))+  " links.")
      time.sleep(1)

    except IndexError:

      descriptions.extend(list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))
      number_of_looks.append(0)
      productS_id.append(0)
      phone_book.append(0)
      avtosalon_name.append(0)
      extras.append(0)
      product_text.append(0)
      location.append(0)
      time_parsed.append(0)
      date_parsed.append(0)

      print("Zeros were parsed in link "+ str(link))







In [ ]:
car_properties_parser()

In [ ]:
###################################################
print(int(len(descriptions)/14))
print(len(number_of_looks))
print(len(productS_id))
print(len(phone_book))
print(len(avtosalon_name))
print(len(extras))
print(len(product_text))
print(len(location))
print(len(time_parsed))
print(len(date_parsed))
print(len(list_of_links))
print(len(dates_times))
print(len(vip_premium))
print(len(loans))
print(len(barters))

######################################################
####Naming columns

first_column_names = []

html = requests.get(list_of_links[0], headers=HEADERS).content
soup = BeautifulSoup(html, "html.parser")

for labels in soup.find_all("li", class_="product-properties-i"):
  for label in labels.find_all("label"):
    first_column_names.append(label.text)

data = pd.DataFrame(np.array(descriptions).reshape(-1, 14))

data.columns = first_column_names

data["Tarix_Saat"] = dates_times
data["Kateqoriya"] = vip_premium
data['Baxışların sayı'] = number_of_looks
data['Məhsulun id-si'] = productS_id
data['Link'] = list_of_links
data['Kredit mümkündür'] = loans
data['Barter mümkündür'] = barters
data['Əlaqə nömrəsi'] = phone_book
data['Avtosalon adı'] = avtosalon_name
data['Ekstra parametrlər'] = extras
data['Məhsulun mətni'] = product_text
data['Yerləşmə məkanı'] = location
data['Parsinq olunan tarix'] = date_parsed
data['Parsinq olunan zaman'] = time_parsed

##########################################################
import gspread_dataframe as gd
import gspread as gs
gc = gs.authorize(GoogleCredentials.get_application_default())

ws_mdb = gc.open("MercedesDataBase").worksheet("sheet_1")
db_data = gd.get_as_dataframe(ws_mdb)

existed_in_db = []

for link in list_of_links:
  if link not in list(db_data['Link']):
    existed_in_db.append(0)
  elif link in list(db_data['Link']):
    existed_in_db.append(1)

data['existed_in_db'] = existed_in_db

############################################################

checkpoint = data
data = data.iloc[::-1].reset_index(drop=True)

############################################################

##Prints sum of not existed cars vs existing in db
data_to_be_added = data[data['existed_in_db']==0]
print("Number of cars Not existed in DB is: " + str(len(data_to_be_added)))
data_existed_in_db = data[data['existed_in_db']==1]
print("Number of cars existed in DB is: " + str(len(data_existed_in_db)))

###############################################################

print("Length of data to be added: " + str(data_to_be_added.shape[0]))

###############################################################

##Adds data which did not exist in db
import gspread_dataframe as gd
import gspread as gs
gc = gs.authorize(GoogleCredentials.get_application_default())

ws_mdb = gc.open("MercedesDataBase").worksheet("sheet_1")
ws_mdb.add_rows(data_to_be_added.shape[0])
gd.set_with_dataframe(worksheet=ws_mdb, dataframe=data_to_be_added, include_index=False, include_column_header=False,
                      row=ws_mdb.row_count+1)

##################################################################

data_existed_in_db_necessary_columns = data_existed_in_db[['Qiymət', 'Tarix_Saat',
       'Kateqoriya', 'Baxışların sayı', 'Məhsulun id-si', 'Link','Əlaqə nömrəsi', 
       'Parsinq olunan tarix', 'Parsinq olunan zaman','existed_in_db']]

##this is to write information for the first time
# ws = gc.open("Mercedes_DB_Updated_info_of_existing_cars").worksheet("sheet_1")
# gd.set_with_dataframe(ws, data_existed_in_db_necessary_columns, resize=True)

#This is to add new data_existed_in_db_necessary_columns to existing database

ws_updated = gc.open("Mercedes_DB_Updated_info_of_existing_cars").worksheet("sheet_1")
ws_updated.add_rows(data_existed_in_db_necessary_columns.shape[0])

gd.set_with_dataframe(worksheet=ws_updated, dataframe=data_existed_in_db_necessary_columns,
                       include_index=False, include_column_header=False,
                       row=ws_updated.row_count+1)


0
0
0
0
0
0
0
0
0
0
0
960
960
960
960


IndexError: ignored